In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
#安装 simple itk
!mkdir /home/aistudio/external-libraries
!pip install SimpleITK -t /home/aistudio/external-libraries
!pip install tqdm -t /home/aistudio/external-libraries
!pip install scikit-image==0.17.2 --no-dependencies -t /home/aistudio/external-libraries
!pip install tifffile>=2019.7.26 -t /home/aistudio/external-libraries
!pip install lxml -t /home/aistudio/external-libraries
! pip install paddlex -i https://mirror.baidu.com/pypi/simple
# networkx>=2.0
# pillow>=4.3.0,!=7.1.0,!=7.1.1
# imageio>=2.3.0

# PyWavelets>=1.1.1

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
#解押文件操作
import os
os.chdir('/home/aistudio/data/data8689')
import subprocess

def excuteCommand(com):
    ex = subprocess.Popen(com, stdout=subprocess.PIPE, shell=True)
    out, err  = ex.communicate()
    status = ex.wait()
    print("cmd in:", com)
    print("cmd out: ", out.decode())
    return out.decode()
dataFileNameList = os.popen('ls').read().split('\n')
print (dataFileNameList)
for dataFile in dataFileNameList:
    if '.zip' in dataFile:
        print('unziping file...')
        unzipResult = excuteCommand('unzip ' + str(dataFile)+ ' > /dev/null')
        # print(unzipResult)
        try:
            os.remove(str(dataFile))
            print('deleting {} successful'.format(dataFile))
        except Exception as e:
            print('deleting {} with error {}'.format(dataFile,str(e)))
# print(aa)

['chestCT_round1_annotation.csv', 'chestCT_round1_testA.zip', 'chestCT_round1_train_part1.zip', 'chestCT_round1_train_part2.zip', 'chestCT_round1_train_part3.zip', 'chestCT_round1_train_part4.zip', 'chestCT_round1_train_part5.zip', '']
unziping file...
cmd in: unzip chestCT_round1_testA.zip > /dev/null
cmd out:  
deleting chestCT_round1_testA.zip successful
unziping file...
cmd in: unzip chestCT_round1_train_part1.zip > /dev/null
cmd out:  
deleting chestCT_round1_train_part1.zip successful
unziping file...
cmd in: unzip chestCT_round1_train_part2.zip > /dev/null
cmd out:  
deleting chestCT_round1_train_part2.zip successful
unziping file...
cmd in: unzip chestCT_round1_train_part3.zip > /dev/null
cmd out:  
deleting chestCT_round1_train_part3.zip successful
unziping file...
cmd in: unzip chestCT_round1_train_part4.zip > /dev/null
cmd out:  
deleting chestCT_round1_train_part4.zip successful
unziping file...
cmd in: unzip chestCT_round1_train_part5.zip > /dev/null
cmd out:  
deleting ch

In [ ]:
#将所有train_part文件夹 放到 train目录里面
#目标文件夹，此处为相对路径，也可以改为绝对路径
os.chdir('/home/aistudio/work')
determination = 'train/'
if not os.path.exists(determination):
    os.makedirs(determination)

#源文件夹路径
dataPath = r'/home/aistudio/data/data8689'
folders= os.listdir(dataPath)
for folder in folders:
    if 'train_part' in folder:
        # dir = path + '/' +  str(folder)
        copyFileResult = excuteCommand('\cp -rf --link {}/* {}/'.format((dataPath+ '/' +  str(folder)),'train'))
        
        # files = os.listdir(dir)
        # for fileName in files:
        #     source = dir + '\\' + str(file)
        #     deter = determination + str(file)
        #     shutil.copyfile(source, deter)

cmd in: \cp -rf --link /home/aistudio/data/data8689/train_part4/* train/
cmd out:  
cmd in: \cp -rf --link /home/aistudio/data/data8689/train_part2/* train/
cmd out:  
cmd in: \cp -rf --link /home/aistudio/data/data8689/train_part1/* train/
cmd out:  
cmd in: \cp -rf --link /home/aistudio/data/data8689/train_part3/* train/
cmd out:  
cmd in: \cp -rf --link /home/aistudio/data/data8689/train_part5/* train/
cmd out:  


In [ ]:
#ct世界坐标转换
import sys
sys.path.append('/home/aistudio/external-libraries')
import numpy as np
import pandas as pd
import SimpleITK as sitk

pd.set_option('display.width', 120)

annotation_file = '/home/aistudio/data/data8689/chestCT_round1_annotation.csv'
df = pd.read_csv(annotation_file, dtype={'seriesuid': int, 'label': int})
# print(df.head())
#遍历并转换坐标
for uid in df['seriesuid'].unique():
    idx = df['seriesuid'] == uid
    mhd_file = './train/{}.mhd'.format(uid)
    itk_image = sitk.ReadImage(mhd_file)
    origin = np.array(itk_image.GetOrigin())
    spacing = np.array(itk_image.GetSpacing())
    df.loc[idx, ['coordX', 'coordY', 'coordZ']] = ((df.loc[idx, ['coordX', 'coordY', 'coordZ']] - origin) / spacing).round()
    df.loc[idx, ['diameterX', 'diameterY', 'diameterZ']] = (df.loc[idx, ['diameterX', 'diameterY', 'diameterZ']] / spacing).round()

df = df.astype('int64')
print(df.head(10))
df.to_csv('./annotation_voxel.csv', index=False)

   seriesuid  coordX  coordY  coordZ  diameterX  diameterY  diameterZ  label
0     636990     214     257      28         31         21          3     32
1     636990     210     231      31         15         20          3     32
2     636990     237     258      34         27         28          6     32
3     636990     158     217      33         13         12          1      1
4     636990     171     200      34         11         15          1      1
5     636990     270     252      36         18         10          1     31
6     696540     172     364      17         53         54          8      5
7     696540     310     282      19         25         18          1      5
8     696540     164     388      24         26          9          1      5
9     696540     126     366      33         16         16          1      5


In [ ]:
#生成全部png 图片 先生成标记 然后反向生成图片 省地方
import numpy as np
import pandas as pd
import os
import sys
sys.path.append('/home/aistudio/external-libraries')
import SimpleITK as sitk
from work import generate_the_image
os.chdir('/home/aistudio/work')
file_paths = ['./train']
save_path = './image_png'
markedFileName = 'train.txt'
for file_path in file_paths:
    files = generate_the_image.get_file_name(file_path)
    images = generate_the_image.get_all_image(file_path, files, save_path, save_image=True,trainFileName=markedFileName)

In [1]:
#生成标记图片
import numpy as np
import pandas as pd
import os
import sys
sys.path.append('/home/aistudio/external-libraries')
import SimpleITK as sitk
from work import lt_annotation
os.chdir('/home/aistudio/work')

sets = ['./train']
data_path = '/home/aistudio/work'
png_image_path = './image_png'
anns_path = '/home/aistudio/data/data8689/chestCT_round1_annotation.csv'

lt_annotation.DoMain(sets,data_path,png_image_path,anns_path)

100%|██████████| 1470/1470 [07:23<00:00,  3.32it/s]


In [ ]:
#标注名字
label_mapper = {
    1: 'nodule',
    5: 'stripe',
    31: 'artery',
    32: 'lymph',
}

In [4]:

# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

In [8]:
#图像增强
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=800, max_size=1333),
    transforms.Padding(coarsest_stride=32),
])

In [ ]:
#转换yolo 及 voc 格式 参考 https://blog.csdn.net/qq_29762941/article/details/80797790

In [23]:
os.chdir('/home/aistudio/work')
train_dataset = pdx.datasets.VOCDetection(
    data_dir='/home/aistudio/work/',
    file_list='./train_list_voc.txt',
    label_list='./labels.txt',
    transforms=train_transforms,
    shuffle=True)
# eval_dataset = pdx.datasets.VOCDetection(
#     data_dir='insect_det',
#     file_list='insect_det/val_list.txt',
#     label_list='insect_det/labels.txt',
#     transforms=eval_transforms)

2020-08-17 17:05:36 [INFO]	Starting to read file list from dataset...
2020-08-17 17:05:53 [INFO]	17167 samples in file ./train_list_voc.txt
creating index...
index created!


In [24]:
! rm -rf ~/log & rm -rf output/faster_rcnn_r50_fpn
! mkdir -p output/faster_rcnn_r50_fpn/vdl_log
! ln -s output/faster_rcnn_r50_fpn/vdl_log ~/log

num_classes = len(train_dataset.labels) + 1
model = pdx.det.FasterRCNN(num_classes=num_classes)
model.train(
    num_epochs=12,
    train_dataset=train_dataset,
    train_batch_size=2,
    # eval_dataset=eval_dataset, 
    learning_rate=0.0025,
    lr_decay_epochs=[8, 11],
    save_interval_epochs=1,
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

2020-08-17 17:12:31,416-INFO: If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!


2020-08-17 17:12:32 [INFO]	Connecting PaddleHub server to get pretrain weights...
[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmp7atj0xl0/ResNet50_cos_pretrained.tar
[==================================================] 100.00%
2020-08-17 17:12:45 [INFO]	Load pretrain weights from output/faster_rcnn_r50_fpn/pretrain/DetResNet50.


2020-08-17 17:12:45,668-WARNING: output/faster_rcnn_r50_fpn/pretrain/DetResNet50.pdparams not found, try to load model file saved with [ save_params, save_persistables, save_vars ]


2020-08-17 17:12:46 [INFO]	There are 265 varaibles in output/faster_rcnn_r50_fpn/pretrain/DetResNet50 are loaded.
2020-08-17 17:13:40 [INFO]	[TRAIN] Epoch=1/12, Step=2/8583, loss=2.028481, loss_cls=1.336804, loss_bbox=3e-06, loss_rpn_cls=0.68968, loss_rpn_bbox=0.001995, lr=0.000837, time_each_step=27.15s, eta=776:41:42
